<a href="https://colab.research.google.com/github/arthur-kt/PPP-Interporal-Converter/blob/main/PPP_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install fredapi  # Install fredapi module
!pip install pppfy  # Install pppfy module


from fredapi import Fred
from pppfy.converter import Converter
import pandas as pd

# Inicialize o conversor do PPPfy e configure a chave do FRED
c = Converter()
fred = Fred(api_key='11cb1e53b57458acaddd29a16e0efe1b')


In [12]:

# Obtenha o CPI dos EUA para ajustar pela inflação
cpi_data = fred.get_series('CPIAUCSL')
cpi_df = cpi_data.to_frame(name='CPI')
cpi_df.index = pd.to_datetime(cpi_df.index)
cpi_df_ano = cpi_df.resample('Y').mean()  # Média anual do CPI

# CPI atual para trazer o valor a 2024
cpi_atual = cpi_df_ano.loc['2024'].values[0]

# Dados de exemplo
dados = {
    'País': ['US', 'US', 'US', 'US','US','GB', 'GB', 'GB', 'GB', 'GB', 'GB', 'GB', 'GB', 'GB', 'GB', 'CN', 'US', 'CN', 'CN', 'CN', 'CN', 'US', 'CN'],  # Usando códigos ISO2 dos países
    'Ano': [2021, 2021, 2021, 2021, 2022, 2021, 2021, 2017, 2017, 2019, 2017, 2019, 2021, 2022, 2022, 2016, 2019, 2016, 2019, 2009, 2020, 2020, 2021],
    'Gasto_Local': [4.46, 2.65, 6.5, 2.5, 2.105, 17, 7.6, 5, 80, 4, 3, 948.1, 560, 650, 520, 16, 291, 5.64, 1861.695, 15, 74, 3.8, 1.147]  # Gasto em moeda local
}

df = pd.DataFrame(dados)


<ipython-input-12-6f5bfeae5433>:5: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  cpi_df_ano = cpi_df.resample('Y').mean()  # Média anual do CPI


In [14]:
# Função para converter gasto local em dólar PPP e ajustar pela inflação dos EUA
def converter_para_dolar_corrente(row):
    # Converta o gasto local para dólares ajustados pelo PPP usando pppfy
    ppp_adjusted = c.get_price_mapping(
        source_country=row['País'],
        source_price=row['Gasto_Local'],
        destination_country="US",
        year=row['Ano']
    )
    gasto_em_dolar_ppp = ppp_adjusted["ppp_adjusted_local_price"]

    # Ajuste pela inflação dos EUA
    cpi_ano_referencia = cpi_df_ano.loc[str(row['Ano'])].values[0]
    gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_ano_referencia)

    return gasto_ajustado

In [13]:
# Aplicar a função para cada linha no DataFrame
df['Gasto_Ajustado_Dolar'] = df.apply(converter_para_dolar_corrente, axis=1)

# Resultado final
print(df[['País', 'Ano', 'Gasto_Local', 'Gasto_Ajustado_Dolar']])

   País   Ano  Gasto_Local  Gasto_Ajustado_Dolar
0    US  2021        4.460   [5.147827313201644]
1    US  2021        2.650  [3.0586866322834876]
2    US  2021        6.500   [7.502438909374592]
3    US  2021        2.500  [2.8855534266825353]
4    US  2022        2.105   [2.249836057139217]
5    GB  2021       17.000  [28.998051159435406]
6    GB  2021        7.600  [12.963834635982886]
7    GB  2017        5.000   [9.318705775840671]
8    GB  2017       80.000  [149.09929241345074]
9    GB  2019        4.000   [7.201877338434628]
10   GB  2017        3.000   [5.591223465504402]
11   GB  2019      948.100   [1707.024976142468]
12   GB  2021      560.000   [955.2299205461075]
13   GB  2022      650.000  [1019.7629731778202]
14   GB  2022      520.000   [815.8103785422561]
15   CN  2016       16.000   [5.227045344030632]
16   US  2019      291.000   [355.9970898005789]
17   CN  2016        5.640  [1.8425334837707976]
18   CN  2019     1861.695   [541.2990353195443]
19   CN  2009       